In [1]:
import difflib
import json
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

FAISS_DIR = "faiss_link_index"

def load_links_from_file(file_path):
    """Loads a dictionary of links from a file, supporting JSON, dictionary, and plain text formats."""
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read().strip()

        try:
            return json.loads(content)
        except json.JSONDecodeError:
            pass

        try:
            evaluated = eval(content)
            if isinstance(evaluated, dict):
                return evaluated
        except Exception:
            pass

        name_link_dict = {}
        for line in content.splitlines():
            if ":" in line:
                key, value = map(str.strip, line.split(":", 1))
                name_link_dict[key] = value

        return name_link_dict if name_link_dict else None

def build_or_load_faiss(name_link_dict):
    """Builds or loads a FAISS index using LangChain."""
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

    if os.path.exists(FAISS_DIR):
        return FAISS.load_local(FAISS_DIR, embeddings=embeddings)

    documents = [
        Document(page_content=key, metadata={"link": value})
        for key, value in name_link_dict.items()
    ]
    db = FAISS.from_documents(documents, embedding=embeddings)
    db.save_local(FAISS_DIR)
    return db

def find_best_match_fuzzy(query, name_link_dict):
    """Fuzzy match fallback if FAISS fails."""
    query = query.lower()
    best_match = difflib.get_close_matches(query, list(name_link_dict.keys()), n=1, cutoff=0.5)
    if best_match:
        return best_match[0], name_link_dict[best_match[0]]
    return None, None

def chatbot(file_path):
    """Runs the AI Link Chatbot with FAISS + Fuzzy fallback."""
    name_link_dict = load_links_from_file(file_path)

    if not name_link_dict:
        print("❌ No valid links found. Please check the file format.")
        return

    print("⚙️ Initializing FAISS...")
    db = build_or_load_faiss(name_link_dict)

    print("🔗 AI Link Chatbot is ready! Type 'exit' to quit.")

    while True:
        user_query = input("\n🤖 Ask for a link: ").strip()
        if user_query.lower() == "exit":
            print("👋 Goodbye!")
            break

        docs = db.similarity_search(user_query, k=1)
        if docs:
            doc = docs[0]
            best_name = doc.page_content
            best_link = doc.metadata.get("link", "Link not found")
            print(f"✅ Best match: {best_name}\n🔗 Link: {best_link}")
        else:
            # Fallback to fuzzy matching
            best_name, best_link = find_best_match_fuzzy(user_query, name_link_dict)
            if best_link:
                print(f"🤖 Fuzzy match: {best_name}\n🔗 Link: {best_link}")
            else:
                print("❌ No relevant link found. Try again!")

# Run the chatbot with the file
chatbot("EDITED_LINKS(1).txt")


⚙️ Initializing FAISS...


/tmp/ipykernel_5517/465275492.py:37: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔗 AI Link Chatbot is ready! Type 'exit' to quit.



🤖 Ask for a link:  rajkori


✅ Best match: "rajkori"
🔗 Link: "https://dseu.ac.in/rajokri/index.html",



🤖 Ask for a link:  exit


👋 Goodbye!
